# Demo Usage of Instrumentation

In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Custom handlers

In [ ]:
from llama_index.core.instrumentation.event_handlers import BaseEventHandler
from llama_index.core.instrumentation.span_handlers import SimpleSpanHandler

In [ ]:
class MyEventHandler(BaseEventHandler):
    @classmethod
    def class_name(cls) -> str:
        """Class name."""
        return "MyEventHandler"

    def handle(self, event) -> None:
        """Logic for handling event."""
        print(event.class_name())
        print(event.dict())
        print("")
        with open("log.txt", "a") as f:
            f.write(str(event))
            f.write("\n")

### Dispatcher


In [ ]:
import llama_index.core.instrumentation as instrument

dispatcher = instrument.get_dispatcher()  # modify root dispatcher

In [ ]:
dispatcher.add_event_handler(MyEventHandler())
dispatcher.span_handler = SimpleSpanHandler()

In [ ]:
dispatcher

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

In [ ]:
qe_dispatcher = instrument.get_dispatcher("llama_index.core.base.query_engine")

In [ ]:
qe_dispatcher

Dispatcher(name='llama_index.core.base.query_engine', event_handlers=[], span_handler=NullSpanHandler(open_spans={}, current_span_id=None), parent_name='root', manager=<llama_index.core.instrumentation.dispatcher.Manager object at 0x14ecaf010>, root_name='root', propagate=True)

In [ ]:
qe_dispatcher.parent

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

In [ ]:
qe_dispatcher.root

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

### Test It Out

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

documents = SimpleDirectoryReader(input_dir="./data/paul_graham").load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
query_result = query_engine.query("Who is Paul?")

QueryStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 36, 55, 642720), 'id_': UUID('0e4ae477-d5c7-4372-a13f-aeb751529b0c')}

RetrievalStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 36, 55, 643303), 'id_': UUID('680de99d-9df0-4f97-a39e-3e9aa48f63fd')}

RetrievalEndEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 36, 55, 824882), 'id_': UUID('113b4318-3be3-4c40-8fd7-a3bf390dc2c8')}

SynthesizeStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 36, 55, 825885), 'id_': UUID('752ff760-3932-44ee-89dd-60c48066b799')}

GetResponseStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 36, 55, 830974), 'id_': UUID('6c5d5a47-449a-418f-be40-80c7d2d3fa43')}

LLMPredictStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 36, 55, 833755), 'id_': UUID('dd8a7de1-9429-43d3-85aa-e54ff0f2a393')}

LLMPredictEndEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 36, 57, 369601), 'id_': UUID('b14e9c88-886f-4984-ae32-61eb9b58b614')}

GetResponseEndEvent
{'t

In [ ]:
dispatcher.span_handler.completed_spans

[SimpleSpan(id_='VectorIndexRetriever._retrieve-95ba7faa-bfd9-455a-b450-d0b4d28c017b', parent_id='BaseRetriever.retrieve-e7248f06-4c84-43ee-a8f7-e6ffdb6c593b', start_time=datetime.datetime(2024, 3, 11, 23, 35, 23, 410243), end_time=datetime.datetime(2024, 3, 11, 23, 35, 23, 566650), duration=0.156407),
 SimpleSpan(id_='BaseRetriever.retrieve-e7248f06-4c84-43ee-a8f7-e6ffdb6c593b', parent_id='RetrieverQueryEngine._query-0ec8d36d-2a63-49e6-8b67-009197c08611', start_time=datetime.datetime(2024, 3, 11, 23, 35, 23, 409762), end_time=datetime.datetime(2024, 3, 11, 23, 35, 23, 567814), duration=0.158052),
 SimpleSpan(id_='LLM.predict-044b5fcf-526a-412c-bd97-c5fb0186cb01', parent_id='Refine.get_response-844d1eb1-ceb6-4e8d-9be7-9d3693ff7c4a', start_time=datetime.datetime(2024, 3, 11, 23, 35, 23, 581012), end_time=datetime.datetime(2024, 3, 11, 23, 35, 25, 142056), duration=1.561044),
 SimpleSpan(id_='Refine.get_response-844d1eb1-ceb6-4e8d-9be7-9d3693ff7c4a', parent_id='CompactAndRefine.get_respo

In [ ]:
query_result = await query_engine.aquery("Who is Paul?")

QueryStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 35, 28, 453426), 'id_': UUID('291b60ee-a057-4382-907e-0bda2880e2d7')}

RetrievalStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 35, 28, 454258), 'id_': UUID('4f65ae0d-0d77-4603-b906-63c8253b05e8')}

RetrievalEndEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 35, 28, 680748), 'id_': UUID('5b9adad3-ff6d-42f8-8fd3-28ea66a8e523')}

SynthesizeStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 35, 28, 681447), 'id_': UUID('f6f2f015-bc87-41e6-8b42-46cfb87b39fa')}

GetResponseStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 35, 28, 685544), 'id_': UUID('1106c2f7-151e-477b-86c8-559e7fbb9af0')}

LLMPredictStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 35, 28, 689004), 'id_': UUID('d7d1618f-9651-47da-9644-cf9ec3b4e025')}

LLMPredictEndEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 35, 29, 911658), 'id_': UUID('fd070268-65c5-465f-b9e8-a7d30f384b5e')}

GetResponseEndEvent
{'t

In [ ]:
dispatcher.span_handler.completed_spans

[SimpleSpan(id_='VectorIndexRetriever._retrieve-95ba7faa-bfd9-455a-b450-d0b4d28c017b', parent_id='BaseRetriever.retrieve-e7248f06-4c84-43ee-a8f7-e6ffdb6c593b', start_time=datetime.datetime(2024, 3, 11, 23, 35, 23, 410243), end_time=datetime.datetime(2024, 3, 11, 23, 35, 23, 566650), duration=0.156407),
 SimpleSpan(id_='BaseRetriever.retrieve-e7248f06-4c84-43ee-a8f7-e6ffdb6c593b', parent_id='RetrieverQueryEngine._query-0ec8d36d-2a63-49e6-8b67-009197c08611', start_time=datetime.datetime(2024, 3, 11, 23, 35, 23, 409762), end_time=datetime.datetime(2024, 3, 11, 23, 35, 23, 567814), duration=0.158052),
 SimpleSpan(id_='LLM.predict-044b5fcf-526a-412c-bd97-c5fb0186cb01', parent_id='Refine.get_response-844d1eb1-ceb6-4e8d-9be7-9d3693ff7c4a', start_time=datetime.datetime(2024, 3, 11, 23, 35, 23, 581012), end_time=datetime.datetime(2024, 3, 11, 23, 35, 25, 142056), duration=1.561044),
 SimpleSpan(id_='Refine.get_response-844d1eb1-ceb6-4e8d-9be7-9d3693ff7c4a', parent_id='CompactAndRefine.get_respo

### Stream Chat

In [ ]:
chat_engine = index.as_chat_engine()

In [ ]:
# response = chat_engine.chat("Tell me a joke.")

In [ ]:
dispatcher.span_handler = SimpleSpanHandler()

In [ ]:
streaming_response = chat_engine.stream_chat("Tell me a joke.")

AgentChatWithStepStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 43, 34, 693024), 'id_': UUID('276d37f1-cfd6-44c6-9d4b-37d027e13252')}

AgentRunStepStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 43, 34, 694414), 'id_': UUID('4a0207bb-6306-4a1e-bdb3-869292a4ea8f')}

StreamChatStartEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 43, 34, 716621), 'id_': UUID('53ca9883-96df-43f1-b4aa-6b165485e761')}

StreamChatEndEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 43, 36, 468939), 'id_': UUID('8f832edd-52ec-4b09-9c7d-4eb836c1e19e')}

AgentToolCallEvent
{'timestamp': datetime.datetime(2024, 3, 11, 23, 43, 36, 470351), 'id_': UUID('91e40227-aea2-447e-bcfa-0aff5b6f0cf9'), 'arguments': '{"input":"Tell me a joke"}', 'tool': ToolMetadata(description='Useful for running a natural language query\nagainst a knowledge base and get back a natural language response.\n', name='query_engine_tool', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>)}



In [ ]:
for token in streaming_response.response_gen:
    print(token, end="")

Why did the computer keep its drinks on the mouse pad? Because it was afraid of spilling them on the hard drive! 😄

### Very Primitive Tree Viz

In [ ]:
# %pip install treelib -q

In [ ]:
from treelib import Node, Tree

tree = Tree()

In [ ]:
sorted_spans = sorted(
    dispatcher.span_handler.completed_spans, key=lambda x: x.start_time
)

In [ ]:
sorted_spans

[SimpleSpan(id_='AgentRunner.stream_chat-f4c8770d-986f-4eaf-b4b7-4fa321a1ee98', parent_id=None, start_time=datetime.datetime(2024, 3, 11, 23, 43, 34, 692304), end_time=datetime.datetime(2024, 3, 11, 23, 43, 38, 221564), duration=3.52926),
 SimpleSpan(id_='AgentRunner._chat-81e3602a-db20-49f9-8b81-7f3b2686de90', parent_id='AgentRunner.stream_chat-f4c8770d-986f-4eaf-b4b7-4fa321a1ee98', start_time=datetime.datetime(2024, 3, 11, 23, 43, 34, 692829), end_time=datetime.datetime(2024, 3, 11, 23, 43, 38, 221534), duration=3.528705),
 SimpleSpan(id_='AgentRunner._run_step-af81c594-3b7c-4dbf-b880-c459b09f4940', parent_id='AgentRunner._chat-81e3602a-db20-49f9-8b81-7f3b2686de90', start_time=datetime.datetime(2024, 3, 11, 23, 43, 34, 694394), end_time=datetime.datetime(2024, 3, 11, 23, 43, 37, 885212), duration=3.190818),
 SimpleSpan(id_='OpenAIAgentWorker._run_step-ea675cc1-ca7c-46bd-a992-5a1901f9675f', parent_id='AgentRunner._run_step-af81c594-3b7c-4dbf-b880-c459b09f4940', start_time=datetime.dat

In [ ]:
for span in sorted_spans:
    tree.create_node(
        tag=f"{span.id_} ({span.duration})",
        identifier=span.id_,
        parent=span.parent_id,
        data=span.start_time,
    )

In [ ]:
print(tree.show(stdout=False, sorting=True, key=lambda node: node.data))

AgentRunner.stream_chat-f4c8770d-986f-4eaf-b4b7-4fa321a1ee98 (3.52926)
└── AgentRunner._chat-81e3602a-db20-49f9-8b81-7f3b2686de90 (3.528705)
    ├── AgentRunner._run_step-af81c594-3b7c-4dbf-b880-c459b09f4940 (3.190818)
    │   └── OpenAIAgentWorker._run_step-ea675cc1-ca7c-46bd-a992-5a1901f9675f (3.190029)
    │       ├── StreamingAgentChatResponse.write_response_to_history-d7b7f879-5655-4edb-9fdd-df3e15eff203 (1.753468)
    │       └── OpenAIAgentWorker._call_function-dd940056-29e1-4f8f-8a65-5f61d2c02cff (1.414462)
    │           └── call_function-dc136ca5-00f7-4e8c-9ebc-e2c09af76e3f (1.414007)
    │               └── BaseQueryEngine.query-5b755a67-8e66-4ecd-bea9-7b381dc7252a (1.413789)
    │                   └── RetrieverQueryEngine._query-36a0b700-6f12-42a8-b1b2-ec74bcccda07 (1.413268)
    │                       ├── BaseRetriever.retrieve-66ff356d-e8f5-4dbe-afcf-302cc564602d (0.21793)
    │                       │   └── VectorIndexRetriever._retrieve-c038a619-d54e-4917-8d90-1d360f